In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from datetime import datetime

# TOC
1. consolidating c_charge_desc
2. creating crime buckets
3. age buckets
4. priors buckest
5. misdemeanor or felony bucket

In [173]:
df_scores = pd.read_csv("compas-scores.csv")
lst = df_scores['c_charge_desc']
print len(lst)

11757


In [174]:
df_scores.head(n=10)

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13
5,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-03-26,Risk of Recidivism,1,Low,2013-03-26
6,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-11-30,Risk of Recidivism,1,Low,2013-11-30
7,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,2,Low,2014-02-19,Risk of Recidivism,6,Medium,2014-02-19
8,9,steven stewart,steven,stewart,2013-08-30,Male,1973-02-25,43,25 - 45,Other,...,NaN,NaN,Risk of Violence,3,Low,2013-08-30,Risk of Recidivism,4,Low,2013-08-30
9,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,1,Low,2014-03-16,Risk of Recidivism,1,Low,2014-03-16


In [175]:
print Counter(df_scores['c_charge_degree'])
print len(Counter(df_scores['c_charge_desc']).most_common())
print Counter(df_scores['c_charge_desc']).most_common()[:40]

Counter({'F': 7232, 'M': 3771, 'O': 754})
532
[('arrest case no charge', 1858), ('Battery', 1811), (nan, 749), ('Possession of Cocaine', 703), ('Grand Theft in the 3rd Degree', 688), ('Driving While License Revoked', 255), ('Driving Under The Influence', 217), ('Felony Battery (Dom Strang)', 174), ('Felony Driving While Lic Suspd', 159), ('Grand Theft (Motor Vehicle)', 139), ('Burglary Unoccupied Dwelling', 132), ('Pos Cannabis W/Intent Sel/Del', 131), ('Possess Cannabis/20 Grams Or Less', 127), ('Aggravated Assault W/Dead Weap', 124), ('Burglary Conveyance Unoccup', 124), ('DUI Property Damage/Injury', 111), ('Possession of Cannabis', 109), ('Aggrav Battery w/Deadly Weapon', 101), ('Felony Petit Theft', 98), ('DUI Level 0.15 Or Minor In Veh', 93), ('Driving License Suspended', 92), ('Uttering a Forged Instrument', 91), ('Resist Officer w/Violence', 90), ('Battery on Law Enforc Officer', 83), ('Poss3,4 Methylenedioxymethcath', 81), ('Possession Of Alprazolam', 79), ('Aggravated Battery

# 1. Consolidating crime categories

In [176]:
for i, j in df_scores.iterrows():
    if df_scores.loc[i, "c_charge_desc"] == 'Aggravated Assault W/dead Weap':
        df_scores.loc[i, "c_charge_desc"] = 'Aggravated Assault W/Dead Weap'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'Grand Theft (motor vehicle)':
        df_scores.loc[i, "c_charge_desc"] = 'Grand Theft (Motor Vehicle)'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'DUI- Enhanced':
        df_scores.loc[i, "c_charge_desc"] = 'DUI - Enhanced'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'DUI- Enhanced':
        df_scores.loc[i, "c_charge_desc"] = 'DUI Property Damage'
        
    elif df_scores.loc[i, "c_charge_desc"] in ['DUI Property Damage', 'DUI/Property Damage/Persnl Inj', 'DUI - Property Damage/Personal Injury']: 
        df_scores.loc[i, "c_charge_desc"] = 'Property Damage/Injury'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'Dealing In Stolen Property':
        df_scores.loc[i, "c_charge_desc"] = 'Dealing in Stolen Property'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'Grand Theft In The 3Rd Degree':
        df_scores.loc[i, "c_charge_desc"] = 'Grand Theft in the 3rd Degree'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'Opert With Susp DL 2ND Offense':
        df_scores.loc[i, "c_charge_desc"] = 'Opert With Susp DL 2nd Offens'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'Offer Agree Secure For Lewd Act':
        df_scores.loc[i, "c_charge_desc"] = 'Offer Agree Secure/Lewd Act'
    
    elif df_scores.loc[i, "c_charge_desc"] in ['Poss Meth/Diox/Meth/Amp (MDMA)', 'Poss 3,4 Methylendioxpyrovlerone']: 
        df_scores.loc[i, "c_charge_desc"] = 'Poss 3,4 MDMA (Ecstasy)'
    
    elif df_scores.loc[i, "c_charge_desc"] in ['Trespass Other Struct/Conve', 'Trespass Struct/Conveyance', 'Tresspass Struct/Conveyance', 'Tresspass in Structure or Conveyance']: 
        df_scores.loc[i, "c_charge_desc"] = 'Trespass Structure/Conveyance'
    
    elif df_scores.loc[i, "c_charge_desc"] == 'Crim Use Of Personal Id Info':
        df_scores.loc[i, "c_charge_desc"] = 'Crim Use of Personal ID Info'
    
    elif df_scores.loc[i, "c_charge_desc"] == 'Crim Attempt/Solic/Consp':
        df_scores.loc[i, "c_charge_desc"] = 'Crim Attempt/Solicit/Consp'
    
    elif df_scores.loc[i, "c_charge_desc"] == 'Agg Fleeing And Eluding':
        df_scores.loc[i, "c_charge_desc"] = 'Agg Fleeing and Eluding'    

In [177]:
len(df_scores['c_charge_desc'].unique())

515

# 2. Creating crime buckets

In [178]:
cats = {}
cats['battery'] = []
cats['assault'] = []
cats['drugs'] = []
cats['driving'] = []
cats['property'] = []
cats['weapon'] = []
cats['sexual'] = []
cats['obstruct'] = []
cats['domestic'] = []
battery = ['batt']
assault = ['assault', 'abuse']
drugs = ['subst', 'cannabi', 'cocaine', 'deliver', 'del ', 'alprazolam', 'pyrro', 'heroin', 
              'hydro','ethylone', 'drug', 'anabolic', 'steroid',
             'tamine', 'meth', 'codeine', 'lsd',
             'buprenorphine', 'lorazepam','carisoprodol', 'benzyl',
             'clonazepam', 'diazepam', 'morph', 'fluor', ',4', 'fentanyl',
             'phentermine', 'butylone']
driving = ['dui', 'driv', 'veh', 'operat', ' dl', 'd.u.i', 'dwi']
prop = ['burg', 'robb', 'arson', 'theft', 'property', 'mischief', 'trespass']
weapon = ['firearm', 'weapon', 'arm', 'wep']
sexual = ['sex', 'prostitu', 'lewd', 'voyeur', 'porn']
obstruct = ['flee', 'forg', 'tamper', 'false', 'obstruct', 'obey', 'resist', ' id']
domestic = ['dom', 'child']
nocat = []
for i in range(len(Counter(df_scores['c_charge_desc']).most_common())):
    charge = Counter(df_scores['c_charge_desc']).most_common()[i][0]
    crime = str(charge).lower()
    if any(d in crime for d in battery):
        cats['battery'].append(charge)
    if any(d in crime for d in assault):
        cats['assault'].append(charge)
    elif any(d in crime for d in drugs):
        cats['drugs'].append(charge)
    elif any(d in crime for d in driving):
        cats['driving'].append(charge)
    elif any(d in crime for d in prop):
        cats['property'].append(charge)
    elif any(d in crime for d in weapon):
        cats['weapon'].append(charge)
    elif any(d in crime for d in sexual):
        cats['sexual'].append(charge)
    elif any(d in crime for d in obstruct):
        cats['obstruct'].append(charge)
    elif any(d in crime for d in domestic):
        cats['domestic'].append(charge)
    else:
        nocat.append((charge, Counter(df_scores['c_charge_desc']).most_common()[i][1]))

In [179]:
print len(cats['drugs'])

105


In [180]:
print len(cats['battery'])
print len(cats['assault'])
print len(cats['drugs'])
print len(cats['driving'])
print len(cats['property'])
print len(cats['weapon'])
print len(cats['sexual'])
print len(cats['obstruct'])
print len(cats['domestic'])
print len(nocat)

35
21
105
42
68
30
30
40
11
168


# 3. Age buckets

In [181]:
np.histogram(df_scores['age'])

a18 = 0
a26 = 0
a34 = 0
a42 = 0
a50 = 0
a58 = 0
last = 0

for a in df_scores['age']:
    if a <= 18:
        a18 += 1
    elif a <= 24:
        a26 += 1
    elif a <= 29:
        a34 += 1
    elif a <= 34:
        a42 += 1
    elif a <= 39:
        a50 += 1
    elif a <= 50:
        a58 += 1
    else:
        last += 1
print a18, a26, a34, a42, a50, a58, last

7 2433 2453 1882 1352 1968 1662


final age buckets: 18, 19-24, 25-29, 30-34, 35-39, 40-50, 50+,

# 4. Priors buckets

In [182]:
print np.histogram(df_scores['juv_fel_count'])
print np.histogram(df_scores['juv_misd_count'])
print np.histogram(df_scores['juv_other_count'])
print np.histogram(df_scores['priors_count'])

(array([11614,   110,    24,     2,     3,     2,     1,     0,     0,
           1]), array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20.]))
(array([11586,   100,    37,    21,     5,     2,     3,     0,     0,
           3]), array([ 0. ,  1.3,  2.6,  3.9,  5.2,  6.5,  7.8,  9.1, 10.4, 11.7, 13. ]))
(array([11550,   172,    26,     2,     3,     2,     1,     0,     0,
           1]), array([ 0. ,  1.7,  3.4,  5.1,  6.8,  8.5, 10.2, 11.9, 13.6, 15.3, 17. ]))
(array([9109, 1398,  603,  374,  139,   74,   46,    6,    6,    2]), array([ 0. ,  4.3,  8.6, 12.9, 17.2, 21.5, 25.8, 30.1, 34.4, 38.7, 43. ]))


(juv felony, misd, other = 1, >=2), priors count >= 1, >=3, >=5, >= 10

# 5. Crime misdemeanor or felony buckets

# 6. jail out - jail in time
cannot finish this until we move all rows with 'O' in the misdemeanor/felony column

In [183]:
o_count = 0
for i, j in df_scores.iterrows():
    degree = df_scores.loc[i, "c_charge_degree"]
    if degree == 'O':
        o_count += 1
    elif degree not in ['M', 'F']:
        print degree
        
print o_count
        

754


# Creating new feature dataframe

In [229]:
df_features = pd.DataFrame(index=range(0,df_scores.shape[0] - o_count), columns = []) 
'''
                           columns = ['prior_battery', 
                             'prior_assault', 
                             'prior_driving', 
                             'prior_drugs', 
                             'prior_property', 
                             'prior_weapons', 
                             'prior_sexual', 
                             'prior_obstruction', 
                             'prior_domestic', 
                             'age_less_18',
                             'age_19_24', 
                             'age_25_29', 
                             'age_30_34', 
                             'age_35_39', 
                             'age_40_50', 
                             'age_50_', 
                             'prior_felony', 
                             'juv_felony_1',
                             'juv_felony_2_',
                             'juv_misd_1',
                             'juv_misd_2_',
                             'juv_other_1',
                             'juv_other_2_',
                             'priors_1',
                             'priors_2_3',
                             'priors_4_5',
                             'priors_6_10',
                             'priors_11_'])
'''

"\n                           columns = ['prior_battery', \n                             'prior_assault', \n                             'prior_driving', \n                             'prior_drugs', \n                             'prior_property', \n                             'prior_weapons', \n                             'prior_sexual', \n                             'prior_obstruction', \n                             'prior_domestic', \n                             'age_less_18',\n                             'age_19_24', \n                             'age_25_29', \n                             'age_30_34', \n                             'age_35_39', \n                             'age_40_50', \n                             'age_50_', \n                             'prior_felony', \n                             'juv_felony_1',\n                             'juv_felony_2_',\n                             'juv_misd_1',\n                             'juv_misd_2_',\n                 

In [230]:
df_scores_no = df_scores[df_scores.c_charge_degree != 'O']

In [231]:
df_scores_no.to_csv('compas-scores-no-O.csv', index=False)

# Populating age buckets

In [232]:
INDEX_AGE = list(df_scores_no.columns).index('age')
ages = np.zeros((df_features.shape[0], 10))

for i in range(df_features.shape[0]):
    age = df_scores_no.iloc[i, INDEX_AGE]
    if age < 18:
        ages[i][0] = 1
    elif age >= 19 and age <=24:
        ages[i][1] = 1
    elif age >= 25 and age <=29:
        ages[i][2] = 1
    elif age >= 30 and age <=34:
        ages[i][3] = 1
    elif age >= 35 and age <=39:
        ages[i][4] = 1
    elif age >= 40 and age <=44:
        ages[i][5] = 1
    elif age >= 45 and age <=49:
        ages[i][6] = 1
    elif age >= 50 and age <=54:
        ages[i][7] = 1
    elif age >= 55 and age <=59:
        ages[i][8] = 1
    elif age >= 60:
        ages[i][9] = 1

In [233]:
df_features['age_less_18'] = ages[:, 0]
df_features['age_19_24'] = ages[:, 1]
df_features['age_25_29'] = ages[:, 2]
df_features['age_30_34'] = ages[:, 3]
df_features['age_35_39'] = ages[:, 4]
df_features['age_40_44'] = ages[:, 5]
df_features['age_45_49'] = ages[:, 6]
df_features['age_50_54'] = ages[:, 7]
df_features['age_55_59'] = ages[:, 8]
df_features['age_60_'] = ages[:, 9]

# Populating Crime buckets

In [234]:
INDEX_CHARGE = list(df_scores_no.columns).index('c_charge_desc')
crimes = np.zeros((df_features.shape[0], 12))

battery = ['batt']
assault = ['assault', 'abuse']
drugs = ['subst', 'cannabi', 'cocaine', 'deliver', 'del ', 'alprazolam', 'pyrro', 'heroin', 
              'hydro','ethylone', 'drug', 'anabolic', 'steroid',
             'tamine', 'meth', 'codeine', 'lsd',
             'buprenorphine', 'lorazepam','carisoprodol', 'benzyl',
             'clonazepam', 'diazepam', 'morph', 'fluor', ',4', 'fentanyl',
             'phentermine', 'butylone']
driving = ['dui', 'driv', 'veh', 'operat', ' dl', 'd.u.i', 'dwi']
prop = ['burg', 'arson', 'theft', 'property', 'mischief', 'trespass', 'fraud', 'stole']
weapon = ['firearm', 'weapon', 'arm', 'wep']
sexual = ['sex', 'prostitu', 'lewd', 'voyeur', 'porn']
obstruct = ['flee', 'forg', 'tamper', 'false', 'obstruct', 'obey', 'resist', ' id']
domestic = ['dom', 'child']
agg = ['agg']
violent = ['rape', 'assault', 'abuse', 'kidnap', 'murder', 'manslaughter']
public_order = ['weapon', 'firearm', 'arm', 'dui', 'driv', 'd.u.i.', 'dwi']

for i in range(df_features.shape[0]):
    crime = str(df_scores_no.iloc[i, INDEX_CHARGE]).lower()
    if any(d in crime for d in battery):
        crimes[i, 0] = 1
    if any(d in crime for d in assault):
        crimes[i, 1] = 1
    if any(d in crime for d in drugs):
        crimes[i, 2] = 1
    if any(d in crime for d in driving):
        crimes[i, 3] = 1
    if any(d in crime for d in prop):
        crimes[i, 4] = 1
    if any(d in crime for d in weapon):
        crimes[i, 5] = 1
    if any(d in crime for d in sexual):
        crimes[i, 6] = 1
    if any(d in crime for d in obstruct):
        crimes[i, 7] = 1
    if any(d in crime for d in domestic):
        crimes[i, 8] = 1
    if any(d in crime for d in agg):
        crimes[i, 9] = 1
    if any(d in crime for d in violent):
        crimes[i, 10] = 1
    if any(d in crime for d in public_order):
        crimes[i, 11] = 1

In [235]:
crimes[:, 0]

array([0., 1., 0., ..., 1., 0., 0.])

In [236]:
df_features['prior_battery'] = crimes[:, 0]
df_features['prior_assault'] = crimes[:, 1]
df_features['prior_driving'] = crimes[:, 2]
df_features['prior_drugs'] = crimes[:, 3]
df_features['prior_property'] = crimes[:, 4]
df_features['prior_weapons'] = crimes[:, 5]
df_features['prior_sexual'] = crimes[:, 6]
df_features['prior_obstruction'] = crimes[:, 7]
df_features['prior_domestic'] = crimes[:, 8]
df_features['prior_agg'] = crimes[:, 9]
df_features['prior_violent'] = crimes[:, 10]
df_features['prior_public_order'] = crimes[:, 11]

In [237]:
df_features['prior_battery']

0        0.0
1        1.0
2        0.0
3        0.0
4        0.0
5        1.0
6        0.0
7        0.0
8        1.0
9        0.0
10       0.0
11       0.0
12       0.0
13       1.0
14       1.0
15       1.0
16       0.0
17       0.0
18       0.0
19       1.0
20       0.0
21       0.0
22       0.0
23       0.0
24       1.0
25       0.0
26       0.0
27       0.0
28       1.0
29       0.0
        ... 
10973    0.0
10974    0.0
10975    0.0
10976    1.0
10977    0.0
10978    0.0
10979    1.0
10980    0.0
10981    0.0
10982    0.0
10983    0.0
10984    1.0
10985    0.0
10986    0.0
10987    1.0
10988    1.0
10989    0.0
10990    0.0
10991    0.0
10992    0.0
10993    0.0
10994    0.0
10995    0.0
10996    1.0
10997    0.0
10998    0.0
10999    1.0
11000    1.0
11001    0.0
11002    0.0
Name: prior_battery, Length: 11003, dtype: float64

# Populating felony/misdemeanor

In [238]:
INDEX_FELONY = list(df_scores_no.columns).index('c_charge_degree')
fel = np.zeros((df_scores_no.shape[0]))

for i in range(df_features.shape[0]):
    f = str(df_scores_no.iloc[i, INDEX_FELONY])
    if f == 'F':
        fel[i] = 1
    elif f != 'M':
        print f

In [239]:
df_features['prior_felony'] = fel
print df_features['prior_battery']

0        0.0
1        1.0
2        0.0
3        0.0
4        0.0
5        1.0
6        0.0
7        0.0
8        1.0
9        0.0
10       0.0
11       0.0
12       0.0
13       1.0
14       1.0
15       1.0
16       0.0
17       0.0
18       0.0
19       1.0
20       0.0
21       0.0
22       0.0
23       0.0
24       1.0
25       0.0
26       0.0
27       0.0
28       1.0
29       0.0
        ... 
10973    0.0
10974    0.0
10975    0.0
10976    1.0
10977    0.0
10978    0.0
10979    1.0
10980    0.0
10981    0.0
10982    0.0
10983    0.0
10984    1.0
10985    0.0
10986    0.0
10987    1.0
10988    1.0
10989    0.0
10990    0.0
10991    0.0
10992    0.0
10993    0.0
10994    0.0
10995    0.0
10996    1.0
10997    0.0
10998    0.0
10999    1.0
11000    1.0
11001    0.0
11002    0.0
Name: prior_battery, Length: 11003, dtype: float64


# Populating priors

In [240]:
INDEX_JF = list(df_scores_no.columns).index('juv_fel_count')
INDEX_JM = list(df_scores_no.columns).index('juv_misd_count')
INDEX_JO = list(df_scores_no.columns).index('juv_other_count')
INDEX_PRIORS = list(df_scores_no.columns).index('priors_count')
jf = np.zeros((df_features.shape[0], 2))
jm = np.zeros((df_features.shape[0], 2))
jo = np.zeros((df_features.shape[0], 2))
priors = np.zeros((df_features.shape[0], 8))


for i in range(df_features.shape[0]):
    jf_count = df_scores_no.iloc[i, INDEX_JF]
    if jf_count == 1:
        jf[i, 0] = 1
    elif jf_count >= 2:
        jf[i, 1] = 1
    
    jm_count = df_scores_no.iloc[i, INDEX_JM]
    if jm_count == 1:
        jm[i, 0] = 1
    elif jm_count >= 2:
        jm[i, 1] = 1
        
    jo_count = df_scores_no.iloc[i, INDEX_JO]
    if jo_count == 1:
        jo[i, 0] = 1
    elif jo_count >= 2:
        jo[i, 1] = 1
    
    priors_count = df_scores_no.iloc[i, INDEX_PRIORS]
    if priors_count == 1:
        priors[i, 0] = 1
    elif priors_count <= 3:
        priors[i, 1] = 1
    elif priors_count <= 5:
        priors[i, 2] = 1
    elif priors_count <= 10:
        priors[i, 3] = 1
    elif priors_count >= 11:
        priors[i, 4] = 1
    elif priors_count == 0:
        priors[i, 5] = 1
    if priors_count >= 6 and priors_count <= 8:
        priors[i, 6]
    if priors_count >= 9 and priors_count <= 10:
        priors[i, 7]

In [241]:
df_features['juv_felony_1'] = jf[:, 0]
df_features['juv_felony_2_'] = jf[:, 1]
df_features['juv_misd_1'] = jm[:, 0]
df_features['juv_misd_2_'] = jm[:, 1]
df_features['juv_other_1'] = jo[:, 0]
df_features['juv_other_2_'] = jo[:, 1]
df_features['priors_1'] = priors[:, 0]
df_features['priors_2_3'] = priors[:, 1]
df_features['priors_4_5'] = priors[:, 2]
df_features['priors_6_10'] = priors[:, 3]
df_features['priors_11_'] = priors[:, 4]
df_features['priors_0'] = priors[:, 5]
df_features['priors_6_8'] = priors[:, 6]
df_features['priors_9_10'] = priors[:, 7]

In [242]:
df_features.head(n=5)

,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,age_60_,...,juv_other_1,juv_other_2_,priors_1,priors_2_3,priors_4_5,priors_6_10,priors_11_,priors_0,priors_6_8,priors_9_10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [243]:
df_features['is_recid'] = np.asarray(df_scores_no['is_recid'])
len(df_scores_no['is_recid'])

11003

In [244]:
print len(df_scores_no['is_recid'])
print list(df_scores_no['is_recid'])[:10]

11003
[0, 1, 1, 0, 0, 0, 1, 0, 0, 0]


In [245]:
df_features['is_recid']

0        0
1        1
2        1
3        0
4        0
5        0
6        1
7        0
8        0
9        0
10       0
11       1
12       0
13       1
14       0
15       0
16       0
17       1
18       1
19       0
20       0
21       1
22       1
23       1
24       1
25       1
26       0
27       1
28       1
29       0
        ..
10973    0
10974    0
10975    1
10976    0
10977    0
10978    0
10979    1
10980    0
10981    0
10982    1
10983    1
10984    1
10985    0
10986    0
10987    0
10988    0
10989    1
10990    0
10991    0
10992    1
10993    0
10994    0
10995    0
10996    0
10997    1
10998    0
10999    1
11000    0
11001    0
11002    0
Name: is_recid, Length: 11003, dtype: int64

In [247]:
df_features['priors_count'] = np.asarray(df_scores_no['priors_count'])
df_features['juv_fel_count'] = np.asarray(df_scores_no['juv_fel_count'])
df_features['juv_misd_count'] = np.asarray(df_scores_no['juv_misd_count'])
df_features['juv_other_count'] = np.asarray(df_scores_no['juv_other_count'])
df_features['age'] = np.asarray(df_scores_no['age'])

In [248]:
cols = list(df_features.columns)
INDEX = cols.index('is_recid')
new_cols = []
new_cols.append(cols[INDEX])
for i in range(len(cols)):
    if i != INDEX:
        new_cols.append(cols[i])

In [249]:
df_features_reorder = df_features[new_cols]
df_features_reorder.head()

,is_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_6_10,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,69
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,34
2,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4,0,0,1,24
3,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,1,0,23
4,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2,0,0,0,43


In [250]:
df_features_reorder['id'] = np.asarray(df_scores_no['id'])

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [251]:
df_features_reorder.to_csv('compas-features-nobinary-id.csv', index = False)

In [204]:
df_full = pd.read_csv("compas-features-1.csv")
df_y = df_full['is_recid']
df_X = df_full.loc[:, df_full.columns != 'is_recid']
cutoff = int(df_X.shape[0]*.7)
df_X_train = df_X.iloc[:cutoff, :]
df_y_train = df_y[:cutoff]
df_X_test = df_X.iloc[cutoff:, :]
df_y_test = df_y[cutoff:]

In [205]:
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(df_X_train, df_y_train)
clf.score(df_X_test, df_y_test)

0.688579218418661

In [24]:
df_temp = pd.read_csv('compas-features-nobinary-id.csv')
df_drugs = pd.read_csv('compas-r-drugs.csv')
df_po = pd.read_csv('compas-r-public-order.csv')
df_violent = pd.read_csv('compas-r-violent.csv')
df_prop = pd.read_csv('compas-r-property.csv')
df_list = [df_temp, df_drugs, df_po, df_violent, df_prop]

In [12]:
print df_temp.columns[:-9]

Index([u'is_recid', u'age_less_18', u'age_19_24', u'age_25_29', u'age_30_34',
       u'age_35_39', u'age_40_44', u'age_45_49', u'age_50_54', u'age_55_59',
       u'age_60_', u'prior_battery', u'prior_assault', u'prior_driving',
       u'prior_drugs', u'prior_property', u'prior_weapons', u'prior_sexual',
       u'prior_obstruction', u'prior_domestic', u'prior_agg', u'prior_violent',
       u'prior_public_order', u'prior_felony', u'juv_felony_1',
       u'juv_felony_2_', u'juv_misd_1', u'juv_misd_2_', u'juv_other_1',
       u'juv_other_2_', u'priors_1', u'priors_2_3', u'priors_4_5',
       u'priors_6_10', u'priors_11_'],
      dtype='object')


In [33]:
df_prop.head()

,is_prop_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,0,0,47,19
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,69,1
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,34,3
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,0,0,1,24,4
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,1,0,23,5


In [48]:
names = ['is_recid', 'is_drug_recid', 'is_po_recid', 'is_violent_recid', 'is_prop_recid']
for col in df_temp.columns[1:-9]:
    results = []
    for i, df in enumerate(df_list):
        temp = df[df[col] == 1]
        temp2 = temp[temp[names[i]] == 1]
        results.append('%0.3f' % (float(temp2.shape[0])/float(df.shape[0])))
        #results.append('%0.3f' % (float(df[df[col] == 1].shape[0])/float(df.shape[0])))
    print  col, '&', results[0], '&', results[1], '&', results[2], '&', results[3], '&', results[4], '\\\\'

age_less_18 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
age_19_24 & 0.088 & 0.018 & 0.011 & 0.022 & 0.022 \\
age_25_29 & 0.082 & 0.017 & 0.017 & 0.021 & 0.013 \\
age_30_34 & 0.058 & 0.012 & 0.012 & 0.013 & 0.012 \\
age_35_39 & 0.035 & 0.006 & 0.006 & 0.009 & 0.006 \\
age_40_44 & 0.020 & 0.002 & 0.004 & 0.005 & 0.004 \\
age_45_49 & 0.018 & 0.003 & 0.002 & 0.003 & 0.004 \\
age_50_54 & 0.017 & 0.003 & 0.002 & 0.003 & 0.005 \\
age_55_59 & 0.011 & 0.002 & 0.001 & 0.002 & 0.003 \\
age_60_ & 0.006 & 0.001 & 0.001 & 0.001 & 0.002 \\
prior_battery & 0.065 & 0.009 & 0.012 & 0.025 & 0.011 \\
prior_assault & 0.010 & 0.002 & 0.002 & 0.003 & 0.001 \\
prior_driving & 0.066 & 0.023 & 0.010 & 0.012 & 0.011 \\
prior_drugs & 0.040 & 0.005 & 0.013 & 0.007 & 0.005 \\
prior_property & 0.061 & 0.008 & 0.008 & 0.011 & 0.022 \\
prior_weapons & 0.014 & 0.003 & 0.002 & 0.005 & 0.003 \\
prior_sexual & 0.002 & 0.000 & 0.000 & 0.000 & 0.000 \\
prior_obstruction & 0.020 & 0.004 & 0.003 & 0.004 & 0.003 \\
prior_domest

In [51]:
for i, df in enumerate(df_list):
    print float(df[df[names[i]] == 1].shape[0])/float(df_temp.shape[0])


0.334272471144
0.0628010542579
0.0568026901754
0.0797055348541
0.0700717986004
